# Microchip ML Models - Model Registry

This notebook trains ML models for the Microchip Intelligence Agent:
- **Revenue Forecasting** - Predict future monthly revenue
- **Customer Churn Prediction** - Classify customers at risk of churning
- **Design Win Conversion** - Predict which designs will go to production

All models are registered to Snowflake Model Registry and can be added as tools to the Intelligence Agent.

## Before You Begin

**Add these packages** in the Packages dropdown (upper right):
- `snowflake-ml-python`
- `scikit-learn`
- `xgboost`
- `matplotlib`

**Database:** MICROCHIP_INTELLIGENCE_TM  
**Schema:** ANALYTICS_TM  
**Warehouse:** MICROCHIP_WH_TM


## Import Required Packages


In [ ]:
# Import Python packages
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

# Import Snowpark ML
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LinearRegression, LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, accuracy_score, roc_auc_score
from snowflake.ml.registry import Registry

print("✅ Packages imported successfully")


## Connect to Snowflake

Get active session and set context to Microchip database.


In [ ]:
# Get active Snowflake session
session = get_active_session()

# Set context
session.use_database('MICROCHIP_INTELLIGENCE_TM')
session.use_schema('ANALYTICS_TM')
session.use_warehouse('MICROCHIP_WH_TM')

print(f"✅ Connected - Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database.Schema: {session.get_fully_qualified_current_schema()}")


---
# MODEL 1: Revenue Forecasting

Predict future monthly revenue using historical order data.


### Prepare Revenue Training Data


In [ ]:
# Get monthly revenue data with features
revenue_df = session.sql("""
SELECT
    DATE_TRUNC('month', order_date)::DATE AS order_month,
    MONTH(order_date) AS month_num,
    YEAR(order_date) AS year_num,
    SUM(order_amount)::FLOAT AS total_revenue,
    COUNT(DISTINCT order_id)::FLOAT AS order_count,
    COUNT(DISTINCT customer_id)::FLOAT AS customer_count,
    AVG(order_amount)::FLOAT AS avg_order_value
FROM RAW_TM.ORDERS_TM
WHERE order_date >= DATEADD('month', -30, CURRENT_DATE())
  AND payment_status = 'COMPLETED'
GROUP BY DATE_TRUNC('month', order_date), MONTH(order_date), YEAR(order_date)
ORDER BY order_month
""")

print(f"Revenue data: {revenue_df.count()} months")
revenue_df.show(5)


### Split Data and Train Revenue Model


In [ ]:
# Train/test split (last 6 months for testing)
train_revenue = revenue_df.filter(F.col("ORDER_MONTH") < F.dateadd("month", F.lit(-6), F.current_date()))
test_revenue = revenue_df.filter(F.col("ORDER_MONTH") >= F.dateadd("month", F.lit(-6), F.current_date()))

# Drop ORDER_MONTH (DATE type not supported in pipeline)
train_revenue = train_revenue.drop("ORDER_MONTH")
test_revenue = test_revenue.drop("ORDER_MONTH")

# Create pipeline
revenue_pipeline = Pipeline([
    ("Scaler", StandardScaler(
        input_cols=["MONTH_NUM", "ORDER_COUNT", "CUSTOMER_COUNT", "AVG_ORDER_VALUE"],
        output_cols=["MONTH_NUM_SCALED", "ORDER_COUNT_SCALED", "CUSTOMER_COUNT_SCALED", "AVG_ORDER_VALUE_SCALED"]
    )),
    ("LinearRegression", LinearRegression(
        label_cols=["TOTAL_REVENUE"],
        output_cols=["PREDICTED_REVENUE"]
    ))
])

# Train model
revenue_pipeline.fit(train_revenue)
print("✅ Revenue forecasting model trained")


### Evaluate and Register Revenue Model


In [ ]:
# Make predictions on test set
test_predictions = revenue_pipeline.predict(test_revenue)

# Calculate metrics
mae = mean_absolute_error(df=test_predictions, y_true_col_names="TOTAL_REVENUE", y_pred_col_names="PREDICTED_REVENUE")
mse = mean_squared_error(df=test_predictions, y_true_col_names="TOTAL_REVENUE", y_pred_col_names="PREDICTED_REVENUE")
rmse = mse ** 0.5

metrics = {"mae": round(mae, 2), "rmse": round(rmse, 2)}
print(f"Model metrics: {metrics}")

# Register model (use different name to avoid conflict with ML Functions)
reg = Registry(session)
reg.log_model(
    model=revenue_pipeline,
    model_name="REVENUE_PREDICTOR",
    version_name="V1",
    comment="Predicts monthly revenue based on historical order patterns using Linear Regression",
    metrics=metrics
)

print("✅ Revenue model registered to Model Registry as REVENUE_PREDICTOR")


---
# MODEL 2: Customer Churn Prediction

Classify customers as likely to churn or not based on behavior patterns.


### Prepare Churn Training Data


In [ ]:
# Get customer features for churn prediction
churn_df = session.sql("""
SELECT
    c.customer_id,
    c.customer_segment,
    c.industry_vertical,
    c.lifetime_value::FLOAT AS lifetime_value,
    c.credit_risk_score::FLOAT AS credit_risk_score,
    -- Recent orders (last 3 months)
    COUNT(DISTINCT CASE WHEN o.order_date >= DATEADD('month', -3, CURRENT_DATE()) 
                   THEN o.order_id END)::FLOAT AS recent_orders,
    -- Historical average
    (COUNT(DISTINCT CASE WHEN o.order_date < DATEADD('month', -3, CURRENT_DATE()) 
                    THEN o.order_id END) / 9.0)::FLOAT AS historical_avg_orders,
    -- Support satisfaction
    AVG(CASE WHEN st.created_date >= DATEADD('month', -6, CURRENT_DATE()) 
        THEN st.customer_satisfaction_score::FLOAT END) AS avg_csat,
    -- Quality issues
    COUNT(DISTINCT qi.quality_issue_id)::FLOAT AS quality_issue_count,
    -- Design wins
    COUNT(DISTINCT CASE WHEN dw.design_win_date >= DATEADD('month', -12, CURRENT_DATE()) 
                   THEN dw.design_win_id END)::FLOAT AS recent_design_wins,
    -- Target: Is churned
    (c.customer_status = 'CHURNED' 
     OR (COUNT(DISTINCT CASE WHEN o.order_date >= DATEADD('month', -3, CURRENT_DATE()) 
                        THEN o.order_id END) = 0 
         AND COUNT(DISTINCT CASE WHEN o.order_date < DATEADD('month', -3, CURRENT_DATE()) 
                            THEN o.order_id END) > 5))::BOOLEAN AS is_churned
FROM RAW_TM.CUSTOMERS_TM c
LEFT JOIN RAW_TM.ORDERS_TM o ON c.customer_id = o.customer_id
LEFT JOIN RAW_TM.SUPPORT_TICKETS_TM st ON c.customer_id = st.customer_id
LEFT JOIN RAW_TM.QUALITY_ISSUES_TM qi ON c.customer_id = qi.customer_id
LEFT JOIN RAW_TM.DESIGN_WINS_TM dw ON c.customer_id = dw.customer_id
GROUP BY c.customer_id, c.customer_segment, c.industry_vertical, c.lifetime_value, c.credit_risk_score, c.customer_status
HAVING COUNT(DISTINCT o.order_id) > 10
""")

print(f"Churn data: {churn_df.count()} customers")
churn_df.show(5)


### Train Churn Classification Model


In [ ]:
# Train/test split (80/20)
train_churn, test_churn = churn_df.random_split([0.8, 0.2], seed=42)

# Drop CUSTOMER_ID and drop original string columns after they'll be encoded
train_churn = train_churn.drop("CUSTOMER_ID")
test_churn = test_churn.drop("CUSTOMER_ID")

# Create pipeline with preprocessing and classification
churn_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["CUSTOMER_SEGMENT", "INDUSTRY_VERTICAL"],
        output_cols=["CUSTOMER_SEGMENT_ENCODED", "INDUSTRY_VERTICAL_ENCODED"],
        drop_input_cols=True,  # Drop original string columns after encoding
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["IS_CHURNED"],
        output_cols=["CHURN_PREDICTION"],
        n_estimators=100,
        max_depth=10
    ))
])

# Train model
churn_pipeline.fit(train_churn)
print("✅ Churn classification model trained")


### Evaluate and Register Churn Model


In [ ]:
# Make predictions
churn_predictions = churn_pipeline.predict(test_churn)

# Calculate metrics
accuracy = accuracy_score(df=churn_predictions, y_true_col_names="IS_CHURNED", y_pred_col_names="CHURN_PREDICTION")
# Note: ROC AUC might need probability scores - using accuracy for now
churn_metrics = {"accuracy": round(accuracy, 4)}
print(f"Churn model metrics: {churn_metrics}")

# Register model (use different name to avoid conflict)
reg.log_model(
    model=churn_pipeline,
    model_name="CHURN_PREDICTOR",
    version_name="V1",
    comment="Predicts customer churn probability using Random Forest based on behavior patterns",
    metrics=churn_metrics
)

print("✅ Churn model registered to Model Registry as CHURN_PREDICTOR")


---
# MODEL 3: Design Win Conversion Prediction

Predict which design wins are likely to convert to production orders.


### Prepare Design Win Conversion Data


In [ ]:
# Get design win features
conversion_df = session.sql("""
SELECT
    dw.design_win_id,
    p.product_family,
    c.customer_segment,
    c.industry_vertical,
    dw.estimated_annual_volume::FLOAT AS estimated_volume,
    dw.competitive_displacement::BOOLEAN AS is_competitive_win,
    -- Has this design gone to production?
    (EXISTS (SELECT 1 FROM RAW_TM.PRODUCTION_ORDERS_TM po 
             WHERE po.design_win_id = dw.design_win_id))::BOOLEAN AS converted_to_production
FROM RAW_TM.DESIGN_WINS_TM dw
JOIN RAW_TM.PRODUCT_CATALOG_TM p ON dw.product_id = p.product_id
JOIN RAW_TM.CUSTOMERS_TM c ON dw.customer_id = c.customer_id
WHERE dw.design_win_date >= DATEADD('month', -24, CURRENT_DATE())
""")

print(f"Design win data: {conversion_df.count()} design wins")
conversion_df.show(5)


### Train Conversion Model


In [ ]:
# Split data
train_conversion, test_conversion = conversion_df.random_split([0.8, 0.2], seed=42)

# Drop DESIGN_WIN_ID (VARCHAR not supported as feature)
train_conversion = train_conversion.drop("DESIGN_WIN_ID")
test_conversion = test_conversion.drop("DESIGN_WIN_ID")

# Create pipeline
conversion_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["PRODUCT_FAMILY", "CUSTOMER_SEGMENT", "INDUSTRY_VERTICAL"],
        output_cols=["PRODUCT_FAMILY_ENC", "CUSTOMER_SEGMENT_ENC", "INDUSTRY_VERTICAL_ENC"],
        drop_input_cols=True,  # Drop original string columns after encoding
        handle_unknown="ignore"
    )),
    ("Classifier", LogisticRegression(
        label_cols=["CONVERTED_TO_PRODUCTION"],
        output_cols=["CONVERSION_PREDICTION"]
    ))
])

# Train
conversion_pipeline.fit(train_conversion)
print("✅ Design win conversion model trained")


### Evaluate and Register Conversion Model


In [ ]:
# Predict on test set
conversion_predictions = conversion_pipeline.predict(test_conversion)

# Calculate accuracy
conv_accuracy = accuracy_score(df=conversion_predictions, 
                                y_true_col_names="CONVERTED_TO_PRODUCTION",
                                y_pred_col_names="CONVERSION_PREDICTION")
conv_metrics = {"accuracy": round(conv_accuracy, 4)}
print(f"Conversion model metrics: {conv_metrics}")

# Register model (use different name to avoid conflict)
reg.log_model(
    model=conversion_pipeline,
    model_name="CONVERSION_PREDICTOR",
    version_name="V1",
    comment="Predicts design win to production conversion using Logistic Regression",
    metrics=conv_metrics
)

print("✅ Conversion model registered to Model Registry as CONVERSION_PREDICTOR")


---
# Verify Models in Registry


In [ ]:
# Show all models in the registry
print("Models in registry:")
reg.show_models()

# Show versions for revenue model
print("\nRevenue model versions:")
reg.get_model("REVENUE_PREDICTOR").show_versions()

# Show versions for churn model  
print("\nChurn model versions:")
reg.get_model("CHURN_PREDICTOR").show_versions()

# Show versions for conversion model
print("\nConversion model versions:")
reg.get_model("CONVERSION_PREDICTOR").show_versions()

print("\n✅ All models registered and ready to add to Intelligence Agent")


---
# Test Model Inference

Test calling each model to make predictions.


In [ ]:
# Test revenue forecast on recent data
revenue_model = reg.get_model("REVENUE_PREDICTOR").default
recent_revenue = revenue_df.limit(3).drop("ORDER_MONTH")
revenue_preds = revenue_model.run(recent_revenue, function_name="predict")
print("Revenue predictions:")
revenue_preds.select("TOTAL_REVENUE", "PREDICTED_REVENUE").show()

# Test churn prediction on sample customers
churn_model = reg.get_model("CHURN_PREDICTOR").default
sample_customers = churn_df.limit(5).drop("CUSTOMER_ID")
churn_preds = churn_model.run(sample_customers, function_name="predict")
print("\nChurn predictions:")
churn_preds.select("IS_CHURNED", "CHURN_PREDICTION").show()

# Test conversion prediction
conversion_model = reg.get_model("CONVERSION_PREDICTOR").default
sample_designs = conversion_df.limit(5).drop("DESIGN_WIN_ID")
conversion_preds = conversion_model.run(sample_designs, function_name="predict")
print("\nConversion predictions:")
conversion_preds.select("CONVERTED_TO_PRODUCTION", "CONVERSION_PREDICTION").show()

print("\n✅ All models tested successfully!")


---
# Next Steps

## Add Models to Intelligence Agent

1. In Snowsight → AI & ML → Agents → MICROCHIP_INTELLIGENCE_AGENT
2. Go to Tools → + Add → Model
3. Add each registered model:
   - **REVENUE_PREDICTOR**
   - **CHURN_PREDICTOR**
   - **CONVERSION_PREDICTOR**

## Example Questions for Agent

- "Forecast revenue for the next quarter using the revenue predictor"
- "Which customers are predicted to churn according to the churn predictor?"
- "Show me design wins with high conversion probability using the conversion predictor"

The models will now be available as tools your agent can use!
